In [1]:
import pandas as pd
import os, json, hashlib, re, orjson
from glob import glob

# Create a hashmap to store the hashed values
func_hashmap = {}
def remove_comments(code):
    """Removes all comments from the given code."""
    code = re.sub(r'//.*', '', code)  # Remove single-line comments
    code = re.sub(r'/\*.*?\*/', '', code, flags=re.DOTALL)  # Remove multi-line comments
    return code
#### read bigvul's train's vuls

# BIGVUL_TRAIN = "../bigvul-cleaned/clean_train.csv"

# df = pd.read_csv(BIGVUL_TRAIN)

# df_filtered = df[(df['target'] == 1) & (df['flaw_line'].str.split().str.len() >= 4)]
 


# for idx, row in df_filtered.iterrows():
#     func_cleaned = remove_comments(row['func_after']).replace(" ", "")  # Remove spaces
#     func_hash = hashlib.sha256(func_cleaned.encode()).hexdigest()  # Hash the function
#     func_hashmap[func_hash] = func_cleaned

# print(len(func_hashmap.keys())) # 4251 - > 4249 (comments removed)
# # Read Devign
# devign_dir = "container_data/train_devign/"
# for filename in os.listdir(devign_dir):
#         if filename.endswith(".c"):
#             file_path = os.path.join(devign_dir, filename)
#             with open(file_path, "r", encoding="utf-8") as f:
#                 content = remove_comments(f.read()).replace(" ", "")  # Remove spaces
#                 content_hash = hashlib.sha256(content.encode()).hexdigest()  # Hash the content
#                 func_hashmap[content_hash] = content

# print(len(func_hashmap.keys())) # 26397 -> 26395 (comments removed)
# # read reveal
# import json
# reveal_dir = "container_data/reveal.jsonl"
# with open(reveal_dir, "r", encoding="utf-8") as f:
#         for line in f:
#             data = json.loads(line)
#             func_cleaned = remove_comments(data["func"]).replace(" ", "")  # Remove spaces
#             # print(func_cleaned)
#             func_hash = hashlib.sha256(func_cleaned.encode()).hexdigest()  # Hash the function
#             func_hashmap[func_hash] = func_cleaned
# print(len(func_hashmap.keys())) # 48780 -> 48778 (comments removed)
# # Read extra cleans
clean_files = glob("./container_data/9_projects/*/Non_vulnerable_functions/*")# Extra clean files from Vulgen's [43]

for file_path in clean_files:
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            content = remove_comments(f.read()).replace(" ", "")  # Remove spaces
            content_hash = hashlib.sha256(content.encode()).hexdigest()  # Hash the content
            func_hashmap[content_hash] = content
    except Exception as e:
        print(e)
        print(file_path)
print(len(func_hashmap.keys())) # 107673 -> 107671 (comments removed)


'utf-8' codec can't decode byte 0xb0 in position 621: invalid start byte
./container_data/9_projects/VLC/Non_vulnerable_functions/puzzle_pce.c_puzzle_rotate_pce.c
58887


In [2]:
def filter_primevul(jsonl_file, func_hashmap, output_file, flaw_lines=False):
    with open(jsonl_file, "rb") as f:
        data = [orjson.loads(line) for line in f]
    if flaw_lines:
        df = pd.DataFrame(data)[['func', 'target', 'idx', 'flaw_line']]
    else:
        df = pd.DataFrame(data)[['func', 'target', 'idx']]
    
    # Remove samples that exist in func_hashmap
    initial_size = len(df)
    df['func_cleaned'] = df['func'].apply(lambda x: remove_comments(x).replace(" ", ""))
    # df['index'] = df['idx']
    # df.rename(columns={"idx": "index"}, inplace=True)
    df['func_hash'] = df['func_cleaned'].apply(lambda x: hashlib.sha256(x.encode()).hexdigest())
    df_filtered = df[~df['func_hash'].isin(func_hashmap)].copy()
    final_size = len(df_filtered)
    # remove comments from PrimeVul's functions
    df_filtered.rename(columns={"idx": "index"}, inplace=True)

    # Print removed sample count
    print(f"Removed {initial_size - final_size} samples.")

    df_filtered['processed_func'] = df_filtered['func'].apply(lambda x: remove_comments(x))


    target_counts = df_filtered['target'].value_counts()
    target_ratio = target_counts[0] / target_counts[1]
    print(f"Ratio of target 0 :{target_counts[0]} to target 1:{target_counts[1]} =  {target_ratio:.2f}")
    cols = ['index', 'processed_func', 'target']
    if flaw_lines:
        cols.append('flaw_line')
    with open(output_file, "wb") as f:
        for record in df_filtered[cols].to_dict(orient='records'):
            # record['idx'] = int(record['idx'])  # safely cast to native int
            f.write(orjson.dumps(record) + b"\n")
    

In [4]:


sets = ["train", "test", "valid"] # 1650, 7, 8 items removed in full dataset, paired -> train 903
for ds in sets:
    print(ds)
    PRIMEVUL_VALID_FILE = f"container_data/primevul_{ds}.jsonl"
    OUTPUT_FILE = f"container_data/primevul_{ds}_cleaned_complete.jsonl"
    filter_primevul(PRIMEVUL_VALID_FILE, func_hashmap, OUTPUT_FILE)

# ratios for full dataset
# train 0 :169345 to target 1:4802 =  35.27
# test 0 :24232 to target 1:549 =  44.14
# val 0 :23347 to target 1:593 =  39.37


train
Removed 100 samples.
Ratio of target 0 :170835 to target 1:4862 =  35.14
test
Removed 0 samples.
Ratio of target 0 :24239 to target 1:549 =  44.15
valid
Removed 0 samples.
Ratio of target 0 :23355 to target 1:593 =  39.38


In [48]:
PRIMEVUL_TRAIN_FILE = f"container_data/primevul_train_paired_with_flawlines.jsonl"
OUTPUT_FILE = f"container_data/primevul_train_cleaned_paired_full.jsonl"
filter_primevul(PRIMEVUL_TRAIN_FILE, func_hashmap, OUTPUT_FILE, flaw_lines=True)

Removed 0 samples.
Ratio of target 0 :3789 to target 1:3789 =  1.00


In [41]:
ds

'valid'

# Diff

In [4]:
import pandas as pd
import orjson
with open("container_data/primevul_train_paired.jsonl", "rb") as f:
    data = [orjson.loads(line) for line in f]
    df = pd.DataFrame(data)[['func', 'target', 'idx']]

df.head()
    

,func,target,idx
0,long ssl_get_algorithm2(SSL *s)\n {\n ...,1,0
1,long ssl_get_algorithm2(SSL *s)\n {\n ...,0,157856
2,gnutls_session_get_data (gnutls_session_t sess...,1,1
3,gnutls_session_get_data (gnutls_session_t sess...,0,157857
4,gnutls_session_get_data (gnutls_session_t sess...,1,2


In [25]:
df['flaw_line'] = ''


In [26]:

def extract_header(func):
    lines = func.strip().split('\n')
    header_lines = []
    for line in lines:
        header_lines.append(line.strip())
        if line.strip().endswith('{'):
            break
    return ' '.join(header_lines)

clean_funcs = {}
for _, row in df[df['target'] == 0].iterrows():
    header = extract_header(row['func'])
    clean_funcs[header] = row['func']

# Process vulnerable functions
for i, row in df[df['target'] == 1].iterrows():
    vul_func = row['func']
    header = extract_header(vul_func)

    clean_func = clean_funcs.get(header)
    if clean_func is None:
        continue  # No match

    # Strip and split
    vul_lines = [line.strip() for line in vul_func.strip().split('\n') if line.strip()]
    clean_lines = [line.strip() for line in clean_func.strip().split('\n') if line.strip()]

    # Extract body (skip header lines)
    vul_body_start = len(extract_header(vul_func).split('\n'))
    clean_body_start = len(extract_header(clean_func).split('\n'))

    vul_body = vul_lines[vul_body_start:]
    clean_body = clean_lines[clean_body_start:]

    # Compute removed lines (in vul but not clean)
    flaw_lines = [line for line in vul_body if line not in clean_body]

    # Save
    df.at[i, 'flaw_line'] = '/~/'.join(flaw_lines)

# Save to JSONL
with open("container_data/primevul_train_paired_with_flawlines.jsonl", "wb") as f:
    for record in df.to_dict(orient='records'):
        f.write(orjson.dumps(record) + b"\n")


In [36]:
multi_lines = 0
empty_flaws=0
print(len(df))
for i in range(len(df)):
    fl = df.loc[i, 'flaw_line']
    if df.loc[i, 'target'] == 1:
        if "/~/" in fl:
            multi_lines += 1
        if fl.strip() == "":
            empty_flaws += 1
        else:
            print(f"FLAW LINES: {fl} \n -------------")
        

print(multi_lines, empty_flaws) 

7578
FLAW LINES: if (TLS1_get_version(s) >= TLS1_2_VERSION && 
 -------------
FLAW LINES: bool pasv_mode_open = false;/~/pasv_mode_open = true;  /* Flag to avoid accept port *//~/} /* err==FTP_OK *//~/if (!pasv_mode_open)   /* Try to use a port command if PASV failed */ 
 -------------
FLAW LINES: add_range(ctx, cmap, high+1, new_high, tree[current].out + high + 1 - tree[current].low, 0, many); 
 -------------
FLAW LINES: if (image->mask)/~/if (gstate->blendmode)/~/if (!image->colorspace)/~/fz_fill_image_mask(ctx, pr->dev, image, &image_ctm,/~/fz_fill_image(ctx, pr->dev, image, &image_ctm, gstate->fill.alpha, &gstate->fill.color_params);/~/if (image->mask)/~/if (gstate->blendmode) 
 -------------
FLAW LINES: buffer = (unsigned char *)gmalloc (width * height * 4); 
 -------------
FLAW LINES: int rlen; 
 -------------
FLAW LINES: int dbufCount, dbufSize, groupCount, *base, *limit, selector,/~/i, j, runPos, symCount, symTotal, nSelectors, byteCount[256];/~/int runCnt = runCnt; /* for comp

In [27]:
print(df.loc[i, 'func'])
print(df.loc[i +1, 'func'])

struct error_obj run_pam_auth(const char *username, char *password) {
    // PAM frees pam_response for us.
    struct pam_response *reply = malloc(sizeof(struct pam_response));
    if (reply == NULL) {
        struct error_obj ret_val;
        ret_val.status = 2;
        ret_val.func_name = "malloc";
        ret_val.error_msg = "Out of memory";
        return ret_val;
    }
    reply->resp = password;
    reply->resp_retcode = 0;

    const struct pam_conv local_conv = { conv_func, reply };
    pam_handle_t *local_auth = NULL;
    int status = pam_start("maddy", username, &local_conv, &local_auth);
    if (status != PAM_SUCCESS) {
        struct error_obj ret_val;
        ret_val.status = 2;
        ret_val.func_name = "pam_start";
        ret_val.error_msg = pam_strerror(local_auth, status);
        return ret_val;
    }

    status = pam_authenticate(local_auth, PAM_SILENT|PAM_DISALLOW_NULL_AUTHTOK);
    if (status != PAM_SUCCESS) {
        struct error_obj ret_val;
        if (stat